# Metaheuristic course - Session 3

In [1]:
import time
import numpy as np
import abc
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from conway import GameOfLife
from IPython.display import clear_output
from metaheuristics import MetaHeuristic

plt.rcParams["animation.html"] = "jshtml"
%matplotlib notebook

## Conway's Game of Live!

To create a new board the most easy way is by using the static method `from_str` where you put the alive cells as `O` and the dead ones as `.` (dot).

> For more ways of creation you can refer to the `GameOfLive` class in the `conway.py` files

The simulation doen't stores the board in memory, only the position of the alive cells, so there are no bounds. The next peace of code will create a board and simulate the next 20 steps of it:

In [2]:
gl = GameOfLife.from_str(
    """
    OO.....
    OO...O.
    ......O
    ....OOO
    """
)

for _ in range(20):
    clear_output()
    print(gl.get_str_board(margin=1), flush=True)
    gl.next_step()
    time.sleep(.1)

..............
.OO...........
.OO...........
..............
..............
..............
..............
..........O...
...........OO.
..........OO..
..............


Let's create a little more fancy animation function:

In [3]:
def animate(gl: GameOfLife, steps: int, ms_per_frame: int = 100):
    fig, ax = plt.subplots()
    ax.matshow(gl.get_board(margin=1), cmap="Blues")

    def _upd(_):
        gl.next_step()
        ax.clear()
        ax.set_aspect('equal')
        ax.set_axis_off()
        ax.matshow(gl.get_board(margin=1), cmap="Blues")

    return animation.FuncAnimation(fig, _upd, interval=ms_per_frame, save_count=steps)

# And test it in a random 100x100 board
gl = GameOfLife.from_random((100, 100))
animate(gl, steps=100, ms_per_frame=50)

<IPython.core.display.Javascript object>

You can check some statistics from the board at any time by using these methods:

- `alive_cells_count`: Number of alive cells.
- `board_size`: Size of the board.
- `board_centroid`: Averaged cells positions (taking the center of the initial setup as reference).
- `board_std`: Standar deviation of cell positions.


In [4]:
print(f"Alive cells: {gl.alive_cells_count()}")
print(f"Board size: {gl.board_size()}")
print(f"Centroid: {gl.board_centroid()}")
print(f"Std of alive cells position: {gl.board_std()}")

# you can allso get the alive cells as an array:
print(gl.cell_array())

Alive cells: 1107
Board size: (143, 139)
Centroid: (1.6693766937669376, 0.6088527551942186)
Std of alive cells position: (32.071523950107384, 34.41541262626051)
[[ -3 -15]
 [ 50 -12]
 [-53 -25]
 ...
 [ -6 -20]
 [-49   9]
 [-14 -24]]


## Optimization inside Conway's Game of Live

Let's now create an objective function to optimize. These functions will recieve an instance of `GameOfLife` as an input and will return a float value to minimize.

For example, let's create a function that computes the farther cell you can get in a simulation from a starting board:

In [5]:
def farther_cell_position(gl: GameOfLife, max_steps: int = 50) -> float:
    for _ in range(max_steps):
        gl.next_step()
    arr = gl.cell_array()
    if len(arr) == 0:
        return -np.inf
    return np.max(np.abs(arr))

Let's test it by creating a glider, which moves diagonally:

In [6]:
# A simple glider
farther_cell_position(GameOfLife.from_str(
    """
    .....
    ..O..
    ...O.
    .OOO.
    .....
    """
))

14

Now, let's see using an oscillator, which doesn't move:

In [7]:
# An oscillator
farther_cell_position(GameOfLife.from_str(
    """
    .....
    ..O..
    ..O..
    ..O..
    .....
    """
))

1

## Time to code!

Using this tools, create an objective function for a given problem and a generative metaheuristic to optimize it

In [24]:
import copy


def generate_solutions(number_solutions, n, m, init_living_cells):
    maximum = 0
    best_sol = ""
    best_init = ""
    maps = []
    for i in range(number_solutions):
        # matrix = [[False if random.randint(0, 100) % 2 == 0 else True for i in range(m)] for j in range(n)]
        while True:
            matrix, d = generate_bool_given_limit(init_living_cells, n, m) # generate a map
            if is_already_done(d, maps): # check if map does not exist
                continue
            maps.append(d)
            print("New map produced:", len(maps), d.keys())
            break

        str_mat = GameOfLife.from_board(matrix)
        init_copy = copy.copy(str_mat.get_str_board())
        result = farther_cell_position(str_mat)
        if result > maximum:
            best_init = init_copy
            maximum = result
            best_sol = str_mat
    print("Best Init:\n", best_init)
    print("Best End:\n", best_sol.get_str_board())
    print("Maximum distance:\n",  maximum)

def generate_bool_given_limit(init_living_cells, n, m):
    d = {}
    matrix = [[False for i in range(m)] for j in range(n)]
    while init_living_cells > 0:
        i = random.randint(0, n-1)
        j = random.randint(0, m-1)
        if (i,j) not in d.keys():
            d[(i, j)] = 0
            matrix[i][j] = True
            init_living_cells -= 1
    return matrix, d

def is_already_done(dic, dict_list):
    return dic in dict_list


generate_solutions(126, 3, 3, 5)

New map produced: 1 dict_keys([(1, 0), (2, 2), (0, 2), (2, 1), (0, 0)])
New map produced: 2 dict_keys([(1, 1), (2, 2), (0, 1), (0, 2), (1, 0)])
New map produced: 3 dict_keys([(1, 0), (2, 2), (1, 2), (0, 0), (1, 1)])
New map produced: 4 dict_keys([(2, 1), (1, 0), (1, 2), (0, 2), (2, 0)])
New map produced: 5 dict_keys([(1, 2), (1, 1), (2, 2), (0, 1), (0, 0)])
New map produced: 6 dict_keys([(0, 1), (0, 2), (1, 1), (2, 1), (2, 0)])
New map produced: 7 dict_keys([(0, 2), (0, 1), (2, 0), (2, 2), (1, 2)])
New map produced: 8 dict_keys([(2, 1), (1, 1), (1, 0), (0, 1), (0, 2)])
New map produced: 9 dict_keys([(2, 1), (0, 1), (0, 0), (1, 1), (2, 2)])
New map produced: 10 dict_keys([(1, 0), (2, 1), (1, 1), (1, 2), (0, 2)])
New map produced: 11 dict_keys([(1, 2), (2, 2), (0, 2), (0, 0), (0, 1)])
New map produced: 12 dict_keys([(0, 1), (0, 2), (0, 0), (1, 0), (2, 2)])
New map produced: 13 dict_keys([(2, 1), (2, 2), (1, 0), (0, 1), (0, 0)])
New map produced: 14 dict_keys([(1, 2), (0, 1), (2, 0), (0, 